In [ ]:
# !nvidia-smi || true
%pip -q install --upgrade pip
!pip install "unsloth>=0.3.0" "torch>=2.0.0" "transformers>=4.36.0" "datasets>=2.14.0" "trl>=0.7.4" "accelerate>=0.24.0" "bitsandbytes>=0.41.0" "scipy>=1.11.0" "click>=8.0.0" "wandb>=0.15.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 21.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 78.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 121.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 79.5 MB/s  0:00:01
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      Successfully uninstalled torchao-0.10.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [unsloth]
ER

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
from trl import SFTTrainer
from transformers import Trainer, TrainingArguments


model, tokenizer = FastLanguageModel.from_pretrained(
model_name="mistralai/Mistral-7B-v0.1",
max_seq_length=2048
)

model = FastLanguageModel.get_peft_model(model)

tokenizer = get_chat_template(
tokenizer,
mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}
)

origdataset = load_dataset("alespalla/chatbot_instruction_prompts", split="train")
pr_dataset = origdataset.select_columns(["prompt", "response"])

# Convert each prompt/response pair into a 2-turn conversation, then to a single 'text' field
def to_chat_text(batch):
    conversations = []
    for p, r in zip(batch["prompt"], batch["response"]):
        conversations.append(
            [
                {"from": "human", "value": p},
                {"from": "gpt", "value": r},
            ]
        )
    texts = [
        tokenizer.apply_chat_template(
            conv, tokenize=False, add_generation_prompt=False
        )
        for conv in conversations
    ]
    return {"text": texts}

dataset = pr_dataset.map(
    to_chat_text,
    batched=True,
    batch_size=100,
    desc="Formatting prompt/response into chat template",
)

trainer = SFTTrainer(
model = model,
tokenizer = tokenizer,
train_dataset = dataset,
dataset_text_field = "text",
dataset_num_proc = 2,
max_seq_length = 2048,
packing = False, # Can make training 5x faster for short sequences.
args = TrainingArguments(
per_device_train_batch_size = 2,
gradient_accumulation_steps = 4,
warmup_steps = 5,
max_steps = 60,
learning_rate = 2e-4,
fp16 = not is_bfloat16_supported(),
bf16 = is_bfloat16_supported(),
logging_steps = 1,
optim = "adamw_8bit",
weight_decay = 0.01,
lr_scheduler_type = "linear",
seed = 3407,
output_dir = "outputs",
report_to = "none", # Use this for WandB etc
),
)
trainer.train()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/258042 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 258,042 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.971300
2,2.186600
3,1.792100
4,1.610200
5,1.461700
6,1.708400
7,1.546100
8,1.453100
9,1.399800
10,1.559700


TrainOutput(global_step=60, training_loss=1.3670972347259522, metrics={'train_runtime': 112.0054, 'train_samples_per_second': 4.286, 'train_steps_per_second': 0.536, 'total_flos': 3046493948510208.0, 'train_loss': 1.3670972347259522, 'epoch': 0.0018601622991606017})

Now that the model is trained, you can use it to generate responses based on new prompts. The following cell shows an example of how to do this.

In [ ]:
# Example inference
prompt = "Translate the phrase 'Good Morning' to French" # Updated prompt

# Apply the chat template to the prompt
# We set add_generation_prompt=True to add the assistant's turn start token
input_text = tokenizer.apply_chat_template([{"from": "human", "value": prompt}], tokenize=False, add_generation_prompt=True)

# Tokenize the input text and include attention mask
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).to(model.device) # Added padding and truncation

# Generate a response, passing the attention mask
outputs = model.generate(inputs.input_ids, max_new_tokens=100, use_cache=True, attention_mask=inputs.attention_mask) # Added attention_mask

# Decode the generated tokens back to text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the original prompt and the generated response
print("Prompt:")
print(prompt)
print("\nGenerated Response:")
print(response)

Prompt:
Translate the phrase 'Good Morning' to French

Generated Response:
<|im_start|>user
Translate the phrase 'Good Morning' to French
<|im_start|>assistant
Bonjour.
